# Building a Database for ZEN

## Build Dataset

The dataset is located at `/nfs/turbo/umms-indikar/shared/projects/RAG/databases/zen2.0`

In [1]:
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [2]:
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [3]:
docsPath='/nfs/turbo/umms-indikar/shared/projects/RAG/papers/zen2.0/'
dbName='ZEN'
dbPath='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/zen2.0/'
HuggingFaceEmbeddingsModel = 'BAAI/bge-base-en-v1.5'
chunk_size=[1000]
chunck_overlap=[50]
v=True

In [4]:
local = os.getcwd()  ## Get local dir
os.chdir(local)      ## shift the work dir to local dir

print('\nWork Directory: {}'.format(local)) if v else None

#%% Phase 1 - Load DB
embeddings_model = HuggingFaceEmbeddings(model_name=HuggingFaceEmbeddingsModel)

print('\nDocuments loading from:', docsPath) if v else None

text_loader_kwargs={'autodetect_encoding': True}


Work Directory: /home/jpic/RAG-DEV/experimental


/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/jpic/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Documents loading from: /nfs/turbo/umms-indikar/shared/projects/RAG/papers/zen2.0/


In [5]:
loader = DirectoryLoader(docsPath,
                         glob="**/*.pdf",
                         loader_cls=PyPDFLoader, 
                         #loader_kwargs=text_loader_kwargs,
                         show_progress=True,
                         )
docs_data = loader.load()

100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


In [6]:
chunk_size = [1000] #Chunk size 
chunk_overlap = [50] #Chunk overlap
for i in range(len(chunk_size)):
    for j in range(len(chunk_overlap)):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size[i],
                                                        chunk_overlap = chunk_overlap[j],
                                                        separators=[" ", ",", "\n", ". "])
        
        data_splits = text_splitter.split_documents(docs_data)
        print('Split Documents: ' + str(len(data_splits)))

        p2_2 = subprocess.run('mkdir  %s/*'%(dbPath+dbName), shell=True)
        _client_settings = chromadb.PersistentClient(path=(dbPath+dbName))

        print('Beginning Chrom Call')
        vectordb = Chroma.from_documents(documents           = data_splits,
                                         embedding           = embeddings_model,
                                         client              = _client_settings,
                                         collection_name     = dbName,
                                         collection_metadata = {"hnsw:space": "cosine"})

        print('Completed Chroma Database: ', dbName) if v else None

mkdir: cannot create directory ‘/nfs/turbo/umms-indikar/shared/projects/RAG/databases/zen2.0/ZEN/*’: No such file or directory


Split Documents: 260
Beginning Chrom Call
Completed Chroma Database:  ZEN


## Test Dataset

In [7]:
from BRAD import brad

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
bot = brad.chatbot(ragvectordb=vectordb)

2024-08-14 15:42:18 INFO semantic_router.utils.logger local


Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD/2024-08-14_15-42-17/log.json. How can I help?


In [19]:
bot.invoke('What settings should I use for high-resolution imaging of live cells?')

2024-08-14 15:42:25,555 - INFO - RAG


BRAD >> 0: 


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Source: ZEN2-Software_Guide_Slidescan.pdf
Content: joystick within the Stage tab to navigate more precisely. The main partof the window contains a LIVE image from the camera.
4Autofocus on the specimen. To do so, you can use the Autofocus button. Ifthe system does not find a focus, you may have to focus manually, e.g., via theslider next to the virtual representation of the z-stack. If the Coarse focus stepwas executed previously and the software does not find a focus, this suggeststhat the focus parameters for the fine focus are not correct.
5Once the specimen is in focus, double-check that the image is not too bright or
too dark and that the white balance is correct. If you need to adju

2024-08-14 15:42:29,736 - INFO - 


route



2024-08-14 15:42:29,737 - INFO - RAG
2024-08-14 15:42:29,737 - INFO - 

PLANNER:

{'module': 'RAG', 'steps': [{'func': 'rag.retrieval', 'multiquery': False, 'similarity': False, 'mmr': True, 'num-docs': 10, 'docs': "[Document(metadata={'page': 87, 'source': '/nfs/turbo/umms-indikar/shared/projects/RAG/papers/zen2.0/ZEN2-Software_Guide_Slidescan.pdf'}, page_content='joystick within the Stage tab to navigate more precisely. The main partof the window contains a LIVE image from the camera.\\n4Autofocus on the specimen. To do so, you can use the Autofocus button. Ifthe system does not find a focus, you may have to focus manually, e.g., via theslider next to the virtual representation of the z-stack. If the Coarse focus stepwas executed previously and the software does not find a focus, this suggeststhat the focus parameters for the fine focus are not correct.\\n5Once the specimen is in focus, double-check that the image is not too bright or\\nto


> Finished chain.

> Finished chain.
The provided context does not specifically mention high-resolution imaging of live cells. However, it does provide some general guidance on autofocus settings, scan settings, and camera settings that might be relevant.

For autofocus settings, the context suggests that the "Best" algorithm might be suitable for certain samples, but it's not always superior to the "Basic" algorithm. The "FFT" algorithm is slower and should only be used for Ring aperture contrast. The "HG 2^8" algorithm is more advanced and focuses on smaller local structures.

For scan settings, the context recommends using the standard parameters for optimal image quality. It also suggests that the camera settings, such as exposure time, white balance, and bit depth, can be adjusted to suit the specific needs of the user.

However, without more specific information about the live cell imaging setup and the desired resolution, it's difficult to provide specific settings recommendati

'The provided context does not specifically mention high-resolution imaging of live cells. However, it does provide some general guidance on autofocus settings, scan settings, and camera settings that might be relevant.\n\nFor autofocus settings, the context suggests that the "Best" algorithm might be suitable for certain samples, but it\'s not always superior to the "Basic" algorithm. The "FFT" algorithm is slower and should only be used for Ring aperture contrast. The "HG 2^8" algorithm is more advanced and focuses on smaller local structures.\n\nFor scan settings, the context recommends using the standard parameters for optimal image quality. It also suggests that the camera settings, such as exposure time, white balance, and bit depth, can be adjusted to suit the specific needs of the user.\n\nHowever, without more specific information about the live cell imaging setup and the desired resolution, it\'s difficult to provide specific settings recommendations.\n\nI would recommend con

In [17]:
vectordb